In [1]:
import numpy as np
# constant size
size = 1000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.1

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [2]:
n_states = 2
n_obs = 2

# emission = np.random.rand(size,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_obs)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
# emission = np.array([[0.5,0.5],[0.5,0.5]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
# transition = np.array([[0.3,0.7],[0.4,0.6]])


forward = np.random.rand(size,n_states)
scale_factors = np.zeros((size))
forward_hat = np.zeros((size,n_states))

backward = np.random.rand(size,n_states)
backward_hat = np.zeros((size,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.0001
max_iter = 100

mu = np.random.rand(n_states)

In [3]:
for ite in range(max_iter):
    forward[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward[0,:])
    forward_hat[0,:] = forward[0,:]/scale_factors[0]
    
    for t in range(size-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        forward[t+1,:] = forward_hat[t+1,:]*np.prod(scale_factors[0:t+1])

    backward[-1,:] = 1
    backward_hat[-1,:] = backward[-1,:]
    for t in reversed(range(size-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],np.transpose(transition))
        backward_hat[t,:] = temp/scale_factors[t+1]
        backward[t,:] = backward_hat[t,:]*np.prod(scale_factors[t+1:-1])


    a = np.zeros((size,n_states))
    b = np.zeros((size,n_states,n_states))
    for i in range(size):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
    for t in range(size-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    # if p - p_old < tol:
    #     break
    p_old = p

p is:-791.4319589098212
p is:-370.77003624631516
p is:-371.28257877649594
p is:-371.3675081969816
p is:-371.3910534647555
p is:-371.3447567452826
p is:-371.24074556287707
p is:-371.11821586134295
p is:-371.0164515588318
p is:-370.9512477622775
p is:-370.91695095946
p is:-370.90132398087235
p is:-370.8948778238349
p is:-370.892391791559
p is:-370.8914772381577
p is:-370.89115356689524
p is:-370.89104384947143
p is:-370.8910091233908
p is:-370.8909996639876
p is:-370.89099816446867
p is:-370.89099881862876
p is:-370.89099980342144
p is:-370.891000628268
p is:-370.8910012200323
p is:-370.8910016174526
p is:-370.89100187552555
p is:-370.89100203995326
p is:-370.8910021435027
p is:-370.891002208214
p is:-370.89100224843463
p is:-370.8910022733305
p is:-370.8910022886904
p is:-370.8910022981409
p is:-370.89100230394183
p is:-370.8910023074945
p is:-370.8910023096664
p is:-370.89100231099223
p is:-370.8910023117995
p is:-370.89100231229094
p is:-370.8910023125894
p is:-370.89100231277024
p is

In [4]:
emission

array([[0.49962351, 0.50037649],
       [0.878     , 0.122     ]])

In [5]:
transition

array([[9.76081113e-24, 1.00000000e+00],
       [1.77044837e-23, 1.00000000e+00]])

In [6]:
from hmmlearn import hmm

In [7]:
model = hmm.CategoricalHMM(n_components=n_states,emissionprob_prior=emission,transmat_prior=transition,startprob_prior=init_prob)


In [8]:
model.fit(data.reshape(-1,1))
hidden_states = model.predict(data.reshape(-1,1))
print("Most likely hidden states:", hidden_states)


/home/watson/.local/lib/python3.10/site-packages/hmmlearn/utils.py:55: RuntimeWarning: invalid value encountered in subtract
  a -= a_lse
Model is not converging.  Current: -inf is not greater than -429.3354678472401. Delta is -inf


ValueError: startprob_ must sum to 1 (got nan)

In [ ]:
data

In [ ]:
model.emissionprob_

In [ ]:
model.transmat_